In [1]:
import tensorflow as ts
import pandas as pd
import numpy as np
import seaborn as sn
import os as os
import glob as glob


## Read in data

In [5]:
data = {}
for f in glob.glob("data/*.csv"):
    print(f)
    data[f] = pd.read_csv(f)

data/2014 data.csv
data/2015 data.csv
data/2016 data.csv
data/2017 data.csv
data/2018 data.csv
data/2019 data.csv
data/2021 data.csv


## Analyze Data

- Look at how Quad 1 Wins/Losses compares to Wins/Losses against Top 25 Teams

## Clean Data

- Apply Standard scaling to all data
- OHE Region
- Drop Conference
- Drop Conference Tournament Champions
- Drop Total Points, Total Opp Points, Total Scoring Differential

### Smaller Model

- Same as above
- Apply PCA to reduce feature count
- Handpick some features

## Create Model

### Neural Network

- Several fully-connected layers

#### Output

##### Win-based

- 7 output nodes, predict number of wins (0-6)
- Apply softmax function at end to get probability

##### Rank-based

- Find the optimal ranking of teams for each year
- 11 output nodes, predict which rank (0-10) the team should be placed in

## Validate Results